# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [45]:
# Imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [3]:
# Constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# Set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

openai = OpenAI()

API key looks good so far


In [35]:
# here is the question; type over this to ask something new
code = """yield from {book.get("author") for book in books if book.get("author")}"""

In [36]:
def get_system_prompt():
    return f"""Act as an AI Code explainer assistant and help explain the provided code, your answer should be in Markdown.
    """

In [37]:
def get_user_prompt(code):
    user_prompt = """Please explain what this code does and why: \n\n"""
    user_prompt += f"{code}"
    return user_prompt

In [40]:
# Get gpt-4o-mini to answer, with streaming
def question_explainer(code):
    """
    """
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages= [
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": get_user_prompt(code)}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
question_explainer(code)

Certainly! Let's break down the provided code snippet step by step.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


#### Key Components:

1. **`yield from`**: 
   - This is a Python feature that allows you to yield all the values from an iterable. It is often used within a generator function to delegate part of its operations to another generator or iterable. In this context, it means that the generator will yield each value produced by the iterable that follows.

2. **Set Comprehension**:
   - The expression is using a set comprehension: `{expression for item in iterable if condition}`. Here, it creates a set containing unique values derived from books that meet a specific condition.
   - Specifically, `{book.get("author") for book in books if book.get("author")}` is the set comprehension in this case.

3. **`book.get("author")`**:
   - This method is used to retrieve the value associated with the key `"author"` from each `book` dictionary. If the key does not exist, `None` is returned instead of raising an error.
   - The code filters out any `book` dictionaries that do not have an `"author"` key or where the author value is `None`.

4. **`if book.get("author")`**:
   - This condition checks if the author exists (i.e., it is not `None` or empty). Only those `book` entries that have a valid author will contribute to the final set.

### What the Code Does

- The entire line of code iterates through a collection, `books`, which is expected to be a list of dictionaries (with each dictionary representing a book).
- For each `book`, it attempts to extract the `author`. If an `author` exists, it is added to a set to ensure uniqueness (duplicate authors will only appear once).
- Finally, it yields all unique authors found in the entire `books` list.

### Example Situation

If `books` is defined as follows:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # Duplicate author should not appear again
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": None}  # Author is None
]


### Result:

Executing the line of code will yield:

- `Author A`
- `Author B`

Since it correctly ignores books with no author and avoids duplicates.

### Summary

- **Purpose**: The code efficiently collects and yields all unique authors from a list of books, while safely handling missing or non-existent author values.
- **Use Case**: This pattern can be particularly useful in scenarios where you want to compile a list of contributors without duplication, such as aggregating authors from a library or a database of publications.

'Certainly! Let\'s break down the provided code snippet step by step.\n\n### Code Breakdown\n\npython\nyield from {book.get("author") for book in books if book.get("author")}\n\n\n#### Key Components:\n\n1. **`yield from`**: \n   - This is a Python feature that allows you to yield all the values from an iterable. It is often used within a generator function to delegate part of its operations to another generator or iterable. In this context, it means that the generator will yield each value produced by the iterable that follows.\n\n2. **Set Comprehension**:\n   - The expression is using a set comprehension: `{expression for item in iterable if condition}`. Here, it creates a set containing unique values derived from books that meet a specific condition.\n   - Specifically, `{book.get("author") for book in books if book.get("author")}` is the set comprehension in this case.\n\n3. **`book.get("author")`**:\n   - This method is used to retrieve the value associated with the key `"author"`

In [58]:
# Get Llama 3.2 to answer
def question_explainer_llama_version(code):
    response = ollama.chat(
        model=MODEL_LLAMA, 
        messages=[
            {"role": "system", "content": get_system_prompt()},
            {"role": "user", "content": get_user_prompt(code)}
        ]
    )
    print(display(Markdown(response['message']['content'])))

In [59]:
question_explainer_llama_version(code)

**Code Explanation**
======================

The given code is a Python expression that utilizes the `yield from` keyword and generator expressions to extract author names from a list of book dictionaries.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's a breakdown of what each part does:

*   `{...}`: This is a dictionary comprehension, which creates a new dictionary containing key-value pairs.
*   `for book in books`: This is the loop variable that iterates over the `books` list.
*   `if book.get("author")`: This condition filters out any book dictionaries that do not have an "author" key.
*   `book.get("author")`: For each book dictionary that passes the filter, this expression retrieves the value associated with the "author" key.
*   `yield from {...}`: The `yield from` keyword is used to delegate the iteration over the generator expression to the caller.

**Why it's used**
----------------

This code is likely used in a context where you want to extract author names from a list of book dictionaries, but only include authors who have a valid name associated with their book. By using `yield from`, we can simplify the iteration process and make the code more readable.

Here's an example use case:

```python
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Jane Smith"}
]

for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```

This will output:

```
John Doe
Jane Smith
```

None
